<a href="https://colab.research.google.com/github/Hamza-Ali0237/HuggingFace-Projects/blob/main/Intermediate/Multilingual%20Translation%20App/Multilingual_Translation_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Libraries
import transformers
from transformers import pipeline, MBartForConditionalGeneration, MBart50TokenizerFast
import torch
import gradio as gr

In [2]:
# Instantiate Model & Tokenizer
model_name = "facebook/mbart-large-50-many-to-many-mmt"

model = MBartForConditionalGeneration.from_pretrained(model_name)
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [3]:
# Translate English To German
article_en = "The quick brown fox jumps over the lazy dog. A new day brings fresh opportunities and challenges. Learn and grow daily."

tokenizer.src_lang = "en_XX"
encoded_en = tokenizer(article_en, return_tensors = 'pt')

generated_tokens = model.generate(
    **encoded_en,
    forced_bos_token_id = tokenizer.lang_code_to_id["de_DE"]
)

output = tokenizer.batch_decode(generated_tokens, skip_special_tokens = True)

print(f'''Original Sentence (English): "{article_en}"\nTranslated Sentence (German): "{output[0]}"''')

Original Sentence (English): "The quick brown fox jumps over the lazy dog. A new day brings fresh opportunities and challenges. Learn and grow daily."
Translated Sentence (German): "Der schnelle braune Fuchs springt über den faulen Hund. Ein neuer Tag bringt frische Chancen und Herausforderungen. Lernen und wachsen täglich."


# Testin On User Input

In [4]:
# Dictionary Of Available Languages
languages = {
    "arabic": "ar_AR",
    "czech": "cs_CZ",
    "german": "de_DE",
    "english": "en_XX",
    "spanish": "es_XX",
    "estonian": "et_EE",
    "finnish": "fi_FI",
    "french": "fr_XX",
    "gujarati": "gu_IN",
    "hindi": "hi_IN",
    "italian": "it_IT",
    "japanese": "ja_XX",
    "kazakh": "kk_KZ",
    "korean": "ko_KR",
    "lithuanian": "lt_LT",
    "latvian": "lv_LV",
    "burmese": "my_MM",
    "nepali": "ne_NP",
    "dutch": "nl_XX",
    "romanian": "ro_RO",
    "russian": "ru_RU",
    "sinhala": "si_LK",
    "turkish": "tr_TR",
    "vietnamese": "vi_VN",
    "chinese": "zh_CN",
    "afrikaans": "af_ZA",
    "azerbaijani": "az_AZ",
    "bengali": "bn_IN",
    "persian": "fa_IR",
    "hebrew": "he_IL",
    "croatian": "hr_HR",
    "indonesian": "id_ID",
    "georgian": "ka_GE",
    "khmer": "km_KH",
    "macedonian": "mk_MK",
    "malayalam": "ml_IN",
    "mongolian": "mn_MN",
    "marathi": "mr_IN",
    "polish": "pl_PL",
    "pashto": "ps_AF",
    "portuguese": "pt_XX",
    "swedish": "sv_SE",
    "swahili": "sw_KE",
    "tamil": "ta_IN",
    "telugu": "te_IN",
    "thai": "th_TH",
    "tagalog": "tl_XX",
    "ukrainian": "uk_UA",
    "urdu": "ur_PK",
    "xhosa": "xh_ZA",
    "galician": "gl_ES",
    "slovene": "sl_SI"
}

In [5]:
# Function To Extract Language Code
def extract_lang_id(lang: str) -> str:
  lang = lang.lower()
  lang_code = languages.get(lang)
  return lang_code

In [6]:
# Modularize The Functionality For Re-usability
def translate_article(article: str, source_lang: str, target_lang: str) -> str:
  source_lang_code = extract_lang_id(source_lang)
  target_lang_code = extract_lang_id(target_lang)

  if source_lang_code is None:
    raise ValueError(f"Source Language '{source_lang}' is not supported.")

  if target_lang_code is None:
    raise ValueError(f"Target Language '{target_lang}' is not supported.")

  tokenizer.src_lang = source_lang_code
  encoded_article = tokenizer(article, return_tensors = "pt")

  generated_tokens = model.generate(
      **encoded_article,
      forced_bos_token_id = tokenizer.lang_code_to_id[target_lang_code]
  )

  output = tokenizer.batch_decode(generated_tokens, skip_special_tokens = True)

  return output[0]

In [7]:
article = input("Enter Text For Translation: ")
source_lang = input("Enter Source Language: ")
target_lang = input("Enter Target Language: ")

output = translate_article(article, source_lang, target_lang)

print(f'Original Sentence {source_lang}: "{article}"\nTranslated Sentence {target_lang}: "{output}"')

Enter Text For Translation: The quick brown fox jumps over the lazy dog. A new day brings fresh opportunities and challenges. Learn and grow daily.
Enter Source Language: English
Enter Target Language: Arabic
Original Sentence English: "The quick brown fox jumps over the lazy dog. A new day brings fresh opportunities and challenges. Learn and grow daily."
Translated Sentence Arabic: "الفرس البني السريع يقفز فوق الكلب المتعطل. يوم جديد يجلبه فرص جديدة وتحديات. تتعلم و تنمو يوميا."


# Gradio UI

In [8]:
def gradio_translate(article, source_lang, target_lang):
    return translate_article(article, source_lang, target_lang)

In [10]:
gr.Interface(
    fn=gradio_translate,
    inputs=[
        gr.Textbox(lines=5, label="Enter Text for Translation"),
        gr.Dropdown(choices=list(languages.keys()), label="Source Language"),
        gr.Dropdown(choices=list(languages.keys()), label="Target Language")
    ],
    outputs=gr.Textbox(label="Translated Text"),
    title="Multilingual Translator",
    description="Translate text between 50+ languages using MBart."
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://da095cb51de266eb53.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
